In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import timm
import numpy as np
import pandas as pd

from torchvision import transforms
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# paths
ZIP_PATH = "/content/drive/MyDrive/data_extracted/faces.zip"
EXTRACT_TO = "/content/drive/MyDrive/data_extracted"

# create clean target folder
# !mkdir -p "$EXTRACT_TO"

# extract
# !unzip -q "$ZIP_PATH" -d "$EXTRACT_TO"

print("Extraction done.")


Using device: cuda
Extraction done.


In [9]:
BASE_DIR  = "/content/drive/MyDrive/data_extracted"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
TEST_DIR  = os.path.join(BASE_DIR, "test")

# class names (folder names = labels)
classes = sorted(os.listdir(TRAIN_DIR))
class_to_idx = {cls: i for i, cls in enumerate(classes)}
print("Classes:", classes)


Classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [13]:
IMG_SIZE = 224

train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # faces often grayscale
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize(256),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [11]:
def build_file_list(root_dir):
    files = []
    labels = []
    for cls in classes:
        cls_dir = os.path.join(root_dir, cls)
        for fname in os.listdir(cls_dir):
            files.append(os.path.join(cls_dir, fname))
            labels.append(class_to_idx[cls])
    return files, labels

train_files, train_labels = build_file_list(TRAIN_DIR)
test_files, test_labels   = build_file_list(TEST_DIR)

print("Train samples:", len(train_files))
print("Test samples :", len(test_files))


def load_image(path, transform):
    img = Image.open(path).convert("L")  # grayscale face
    return transform(img)


Train samples: 28250
Test samples : 7178


In [18]:
# MANUAL BATCH LOADER
def get_batch(files, labels, transform, batch_size, start):
    imgs, labs = [], []
    end = min(start + batch_size, len(files))

    for i in range(start, end):
        imgs.append(load_image(files[i], transform))
        labs.append(torch.tensor(labels[i]))

    return (
        torch.stack(imgs).to(device),
        torch.stack(labs).to(device)
    )

# MODEL (ResNet18)
num_classes = len(classes)

model = timm.create_model("resnet18", pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss()

In [ ]:
# TRAIN & EVAL FUNCTIONS

def train_epoch(files, labels, transform, optimizer, batch_size=32):
    model.train()
    correct = total = 0

    for i in range(0, len(files), batch_size):
        imgs, labs = get_batch(files, labels, transform, batch_size, i)

        optimizer.zero_grad()
        out = model(imgs)
        loss = criterion(out, labs)
        loss.backward()
        optimizer.step()

        preds = out.argmax(1)
        correct += (preds == labs).sum().item()
        total += labs.size(0)

    return correct / total


def evaluate(files, labels, transform, batch_size=32):
    model.eval()
    preds_all, labels_all = [], []

    with torch.no_grad():
        for i in range(0, len(files), batch_size):
            imgs, labs = get_batch(files, labels, transform, batch_size, i)
            out = model(imgs)
            preds_all.append(out.argmax(1).cpu())
            labels_all.append(labs.cpu())

    return (
        torch.cat(labels_all).numpy(),
        torch.cat(preds_all).numpy()
    )


# PARTIAL FINE-TUNING (classifier only)

for p in model.parameters():
    p.requires_grad = False
for p in model.fc.parameters():
    p.requires_grad = True

optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)

print("\nPartial Fine-Tuning")
for epoch in range(3):
    acc = train_epoch(train_files, train_labels, train_transform, optimizer)
    print(f"Epoch {epoch+1}: Train Acc = {acc:.4f}")


In [ ]:


for p in model.parameters():
    p.requires_grad = True

optimizer = optim.Adam(model.parameters(), lr=5e-5)

print("\nFull Fine-Tuning")
for epoch in range(5):
    acc = train_epoch(train_files, train_labels, train_transform, optimizer)
    print(f"Epoch {epoch+1}: Train Acc = {acc:.4f}")

# FINAL TEST METRICS

y_true, y_pred = evaluate(test_files, test_labels, test_transform)

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=classes))

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))